In [1]:
import pandas as pd
import numpy as np

In [2]:
business = pd.read_json("yelp_academic_dataset_business.json", lines=True)

Sneak peek into 2 first rows to realise what we've got.

In [3]:
business.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."


Create 2 bool arrays to finaly get only Austing AND Texas entries.

In [4]:
austin_tx_bool = np.array(business.city == 'Austin') * np.array(business.state == 'TX')
austin_tx = business[austin_tx_bool]

In [5]:
# austin_tx.to_json('austin_tx.json')

Drop unimportant columns.

In [6]:
# austin_tx = austin_tx.drop(['city', 'state', 'business_id', 'address'], axis=1)
austin_tx = austin_tx.drop(['city', 'state', 'business_id'], axis=1)

We have a lot of nan values, let's see how many.

In [7]:
austin_tx.isna().sum()

name               0
address            0
postal_code        0
latitude           0
longitude          0
stars              0
review_count       0
is_open            0
attributes      2269
categories        17
hours           3477
dtype: int64

Let's drop the nans.

In [8]:
dropped = austin_tx.shape[0]
austin_tx.dropna(inplace=True)
dropped -= austin_tx.shape[0]
print("Dropped:", dropped, "rows with nan values.") #5070

Dropped: 5070 rows with nan values.


In [9]:
# austin_tx.to_json('austin_tx_nona.json')

Let's peek to first 2 rows to see what we've got.

In [10]:
austin_tx.head(2)

,name,address,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
8,Lane Wells Jewelry Repair,"7801 N Lamar Blvd, Ste A140",78752,30.346169,-97.711458,5.0,30,1,"{'RestaurantsPriceRange2': '1', 'ByAppointment...","Shopping, Jewelry Repair, Appraisal Services, ...","{'Monday': '12:15-17:0', 'Tuesday': '12:15-17:..."
9,Capital City Barber Shop,"615 W Slaughter Ln, Ste 113",78748,30.172706,-97.799920,4.0,5,0,"{'BusinessAcceptsCreditCards': 'False', 'Resta...","Barbers, Beauty & Spas","{'Monday': '9:0-17:0', 'Tuesday': '9:0-19:0', ..."


## Function to add distances from given place to the dataframe.

Clear address column from strange informations, this fix_adress() is suited for austin_tx database. New one may be required for Berlin.

In [11]:
def fix_address(address):
    return (address.split(sep=', '))[0]
austin_tx['address'] = austin_tx['address'].apply(fix_address)

In [12]:
austin_tx.head(2)

,name,address,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
8,Lane Wells Jewelry Repair,7801 N Lamar Blvd,78752,30.346169,-97.711458,5.0,30,1,"{'RestaurantsPriceRange2': '1', 'ByAppointment...","Shopping, Jewelry Repair, Appraisal Services, ...","{'Monday': '12:15-17:0', 'Tuesday': '12:15-17:..."
9,Capital City Barber Shop,615 W Slaughter Ln,78748,30.172706,-97.799920,4.0,5,0,"{'BusinessAcceptsCreditCards': 'False', 'Resta...","Barbers, Beauty & Spas","{'Monday': '9:0-17:0', 'Tuesday': '9:0-19:0', ..."


### Actual function to use in our work.

In [13]:
# conda install -c conda-forge geopy
from geopy.geocoders import Nominatim
from geopy.distance import distance

# proceed with cauction!!!
def distance_from(dataframe, addresses_column_name, given_location, postifx="", inplace=False):
    """
    dataframe - pandas.DataFrame object
    addresses_column_name - name of the column in which we can find addresses
    given_location - tuple with latitude and longitude of a location from which we're counting the distance

    postfix - what should I add to every address to make sure it's what we need. example: city/country
    inplace - check it if you want to change the given dataframe. default False

    returned value - pd.Series distances in kilometers
    """
    geolocator = Nominatim(user_agent="Strive School YMVC Project")

    number_of_rows = dataframe.shape[0]
    addresses_df = dataframe[addresses_column_name]

    gps_addresses = np.zeros(shape=number_of_rows)

    # this loop can take a looooong time.
    for row_nr in np.arange(0, number_of_rows):
        dist = np.nan
        try:
            loc = geolocator.geocode(addresses_df.iloc[row_nr] + postifx)
            dist = distance(given_location, (loc.latitude, loc.longitude)).kilometers
        except Exception as e:
            print(e)
        gps_addresses[row_nr] = dist

    if inplace:
        # we put a new column before the "address" column
        dataframe.insert(dataframe.columns.get_loc(addresses_column_name), 'distance_km', gps_addresses)
    else:
        return pd.Series(gps_addresses, name='distance_km')

Example of use with inplace=False

In [14]:
# temporary dataframe
head_2 = austin_tx.head(2)

# some coordinates
some_place = (30.274773446583634, -97.74038126660496)

pd_series_distance = distance_from(head_2, 'address', some_place, postifx=" Austin Texas", inplace=False)


In [15]:
# distances stored outside the given dataframe
pd_series_distance

0     8.258619
1    12.785982
Name: distance_km, dtype: float64

In [16]:
# dataframe unchanged
head_2

,name,address,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
8,Lane Wells Jewelry Repair,7801 N Lamar Blvd,78752,30.346169,-97.711458,5.0,30,1,"{'RestaurantsPriceRange2': '1', 'ByAppointment...","Shopping, Jewelry Repair, Appraisal Services, ...","{'Monday': '12:15-17:0', 'Tuesday': '12:15-17:..."
9,Capital City Barber Shop,615 W Slaughter Ln,78748,30.172706,-97.799920,4.0,5,0,"{'BusinessAcceptsCreditCards': 'False', 'Resta...","Barbers, Beauty & Spas","{'Monday': '9:0-17:0', 'Tuesday': '9:0-19:0', ..."


Example use with inplace=True

In [17]:
distance_from(head_2, 'address', some_place, postifx=" Austin Texas", inplace=True)
# returned value: None

In [18]:
# head_2 has new column: distance_km
head_2

,name,distance_km,address,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
8,Lane Wells Jewelry Repair,8.258619,7801 N Lamar Blvd,78752,30.346169,-97.711458,5.0,30,1,"{'RestaurantsPriceRange2': '1', 'ByAppointment...","Shopping, Jewelry Repair, Appraisal Services, ...","{'Monday': '12:15-17:0', 'Tuesday': '12:15-17:..."
9,Capital City Barber Shop,12.785982,615 W Slaughter Ln,78748,30.172706,-97.799920,4.0,5,0,"{'BusinessAcceptsCreditCards': 'False', 'Resta...","Barbers, Beauty & Spas","{'Monday': '9:0-17:0', 'Tuesday': '9:0-19:0', ..."


## Function for indexing dataframes based on strings/string parts that are contained in given column.
(This function is just a mechanism we can use for boolean-indexing the dataframe since it returns boolean array.)

In [19]:
def contains(column, word):
    """
    Usage:
    Pass a column and a word u want to find in the cells, to get a np.array of type bool.
    Afterwards u can pass it as an argument of [] selection mechanism. Examples:

    austin_tx[ contains(austin_tx.categories, "Shopping") ]
    austin_tx[ contains(austin_tx.categories, "Shopping") | contains(austin_tx.categories, "Hotels") ]      # and
    austin_tx[ contains(austin_tx.categories, "Shopping") & contains(austin_tx.categories, "Watches") ]     # or

    Works for all columns that return strings. hours and attributes too.
    """
    return np.array([(word in vals) for vals in column])

Example: get all Shops that do Watches. :)

In [20]:
(austin_tx[ contains(austin_tx.categories, "Shopping") & contains(austin_tx.categories, "Watches") ]).head(2)

,name,address,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
103,Bernard Watch Co.,3834 Spicewood Springs Rd,78759,30.366473,-97.749946,4.5,80,1,"{'RestaurantsPriceRange2': '3', 'BusinessAccep...","Watches, Shopping","{'Monday': '0:0-0:0', 'Tuesday': '10:0-13:30',..."
2710,Villarreal - Designers of Exquisite Jewelry,7600 Burnet Rd,78757,30.353134,-97.733745,4.5,26,1,"{'BusinessParking': '{'garage': False, 'street...","Local Services, Appraisal Services, Shopping, ...","{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'..."


## Function that returns all categories and how much of each there is.

Clear categories column from strange informations, this fix_category() is suited for austin_tx database. New one may be required for Berlin.

In [21]:
def fix_category(category):
    return category.replace(", &", " &")
austin_tx['categories'] = austin_tx['categories'].apply(fix_category)

### Actual function

In [22]:
def get_categories_dict(category_column):
    categories_counts = {}
    for things in category_column:
        for thing in things.split(sep=", "):    
            if thing in categories_counts.keys():
                categories_counts[thing] += 1
            else:
                categories_counts[thing] = 1
    return categories_counts

Example usage:

In [23]:
categories_counts = get_categories_dict(austin_tx['categories'])

I use the loop below to just show first 3 entries.

In [24]:
# few example entries in "categories_counts"
for idx, key in enumerate(categories_counts):
    if idx < 3:
        print("Category name:", key, "\nNumber of entries with this type:", categories_counts[key], "\n")
    else:
        break

Category name: Shopping 
Number of entries with this type: 2920 

Category name: Jewelry Repair 
Number of entries with this type: 42 

Category name: Appraisal Services 
Number of entries with this type: 18 



In [25]:
cc_df = pd.DataFrame.from_dict(categories_counts, orient='index', columns=['counts'])
cc_df.reset_index(level=0, inplace=True)
cc_df.rename(columns={'index':'categories'}, inplace=True)

# let's say the significant ones are those which are more than 1% of the maximum value
significtant = cc_df.counts.max() * 0.01
significant_cathegories = cc_df[cc_df.counts > significtant]

significant_cathegories['average_stars'] = [austin_tx[contains(austin_tx.categories, key)].stars.mean() for key in significant_cathegories['categories']]


<ipython-input-25-fdc704b68475>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  significant_cathegories['average_stars'] = [austin_tx[contains(austin_tx.categories, key)].stars.mean() for key in significant_cathegories['categories']]


In [28]:

significant_cathegories

,categories,counts,average_stars
0,Shopping,2920,3.930479
3,Local Services,1840,3.962500
4,Jewelry,227,4.196507
7,Barbers,189,4.253968
8,Beauty & Spas,1876,4.224947
...,...,...,...
635,Hot Dogs,58,3.913793
636,Party Equipment Rentals,63,4.492063
674,Pet Stores,53,4.301887
746,French,51,3.971154


In [26]:
# uncomment the line below and run this cell if you wanna see EVERYTHING
# categories_counts